# Uses interaction to push triples to the brain and query it

This notebook assumes you already understand the following notebooks:

* lets-chat.ipynb
* roboGrasp-api.ipynb

In this notebook, we will combine the interaction modeled through EMISSOR with the interaction throught *capsules* with the BRAIN. As auxiliary modules, we will use the *cltl-knowledgeextraction* and *cltl-language-generation*. The *cltl-knowledgeextraction* will extract triples from the utterances either for posting or for querying. In the former case, it also extracts the source perspective from the text. The triples and source perspectives are represented in enriched capsules. We will also use a replier that is include in the *cltl-languagegeneration* package. This replier transfers the response from the BRAIN into natural language expressions and possibly gestures. 

In order to connect to the EMISSOR scenario, we need to align the meta properties of the scenario with the meta data in the capsules. However,  the *cltl-knowledgeextraction* uses a similar data object to keep track of the conversation history. This is a Chat object that needs to be created. Through this Chat object, we keep track of what was said before and deal with coreference to earlier utterances.

Combining EMISSOR and the BRAIN, we can model the interaction in a infite while loop where we go through the following steps:

* We create an EMISSOR scenario at the start of the interaction and a corresponding Chat object to keep track of the dialogue history
* while the user does not stop:
    * create TextSignals for each utterance from the user and store these in an EMISSOR scenario
    * add the utterance also to the Chat instance
    * process the latest utterance in the Chat object using the *cltl-knowledgeextraction* to get triples and perspectives
    * create a capsule from the triples, perspectives and the text signal meta data
    * post the triple to the BRAIN
    * get the answer or throughts as a response from the BRAIN
    * verbalise the answer or thoughts
    * create a new TextSignal for the system response and add it to the EMISSOR scenario
* When the user stops, we save the scenario in the EMISSOR format.



Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [9]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

### Import the chatbot utility functions

In [11]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [12]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-12-09-10:29:02 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [13]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-12-09 10:29:21,061 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-12-09 10:29:26,321 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-12-09 10:29:29,189 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-12-09 10:29:29,192 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-12-09 10:29:29,194 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-12-09 10:29:29,196 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-12-09 10:29:29,634 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [14]:
replier = LenkaReplier()

2021-12-09 10:29:31,504 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Establish the speaker identity

In [15]:
#### Small sequence to learn name of speaker
initial_prompt = "Hello, I am "+AGENT
print(AGENT + ": " + initial_prompt)


initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, initial_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario_ctrl, AGENT)
HUMAN_ID = HUMAN_NAME  ### Hack because we cannot force the namespace through capsules, name and identity are the same till this is fixed

print("Name:", HUMAN_NAME)
print("Id:", HUMAN_ID)
                
capsule = c_util.scenario_utterance_to_capsule(scenario_ctrl,place_id,location, textSignal,HUMAN_ID,AGENT,"know", HUMAN_ID)

name_thoughts = my_brain.update(capsule, reason_types=True, create_label=True)

#pprint.pprint(capsule)

Leolani2: Hello, I am Leolani2
Leolani2: Good to see you! I've told you my name, but what about yours? Stranger?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


Name: Piek
Id: Piek
2021-12-09 10:29:51,523 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-12-09 10:29:52,258 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of Piek to: None
2021-12-09 10:29:52,298 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: leolani2_know_piek [person_->_])
2021-12-09 10:29:52,384 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-12-09 10:29:55,537 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: piek on December,2021 about UNDERSPECIFIED
2021-12-09 10:29:55,591 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-friends-with person - 15 gaps as object: e.g. write-by book
2021-12-09 10:29:55,641 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 25 gaps as su

## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [16]:
chat = Chat(HUMAN_ID)

2021-12-09 10:30:07,659 -     INFO - cltl.triple_extraction.api.Chat (Piek)              000 - << Start of Chat with Piek >>


## Start the interaction

In [17]:
print_details=False


#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, initial_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
    scenario_ctrl.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario_ctrl,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain,
                           print_details)

    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, reply, AGENT)
    scenario_ctrl.append_signal(textSignal)


Leolani2: Tell me anything, I love learning things



 I am from Weesp


Piek: I am from Weesp
2021-12-09 10:30:20,010 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-12-09 10:30:20,012 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-12-09 10:30:20,018 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-12-09 10:30:21,527 -     INFO - cltl.triple_extraction.api.Chat (Piek)              001 -       Piek: "I am from Weesp"
initial triple: {'subject': 'I', 'predicate': 'am', 'object': 'from-Weesp'}
2021-12-09 10:30:22,848 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: piek_be-from_weesp [agent_->_person])
2021-12-09 10:30:25,437 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: piek on December,2021 about POSITIVE
2021-12-09 10:30:25,538 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. born-in location - 15 gaps as object: e.g. like-by interest
2021-12-09 10:30:25,592 -     INFO - 


 I work at the VU


Piek: I work at the VU
2021-12-09 10:30:36,876 -     INFO - cltl.triple_extraction.api.Chat (Piek)              002 -       Piek: "I work at the VU"
initial triple: {'subject': 'I', 'predicate': 'work', 'object': 'at-the-VU'}
2021-12-09 10:30:37,633 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of VU to: country
2021-12-09 10:30:37,694 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: piek_work-at_the-vu [agent_->_country])
2021-12-09 10:30:37,790 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-12-09 10:30:40,191 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: piek on December,2021 about POSITIVE
2021-12-09 10:30:40,291 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. like-by agent - 15 gaps as object: e.g. cook-by food
2021-12-09 10:30:40,343 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 0 gaps 

...

Piek: What do I like
2021-12-09 10:31:38,863 -     INFO - cltl.triple_extraction.api.Chat (Piek)              008 -       Piek: "What do I like"
2021-12-09 10:31:38,867 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: piek_like_? [agent_->_])
Leolani2: you told me you like cats and that you like dogs



 stop


Piek: stop
2021-12-09 10:31:46,329 -     INFO - cltl.triple_extraction.api.Chat (Piek)              009 -       Piek: "stop"


Couldn't parse input


Leolani2: Why?


## Switch speaker

Reset the identity

In [ ]:
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

In [18]:
#### Small sequence to learn name of speaker
initial_prompt = "Hello, I am "+AGENT
print(AGENT + ": " + initial_prompt)


initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, initial_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario_ctrl, AGENT)
HUMAN_ID = HUMAN_NAME  ### Hack because we cannot force the namespace through capsules, name and identity are the same till this is fixed

print("Name:", HUMAN_NAME)
print("Id:", HUMAN_ID)
                
capsule = c_util.scenario_utterance_to_capsule(scenario_ctrl,place_id,location, textSignal,HUMAN_ID,AGENT,"know", HUMAN_ID)

name_thoughts = my_brain.update(capsule, reason_types=True, create_label=True)
chat = Chat(HUMAN_ID)

Leolani2: Hello, I am Leolani2
Leolani2: Nice to see you! I would like to know your name! Piek?



 Fred


Fred
Leolani2: So your name is Fred?



 yes


Name: Fred
Id: Fred
2021-12-09 10:33:02,973 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-12-09 10:33:03,639 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of Fred to: None
2021-12-09 10:33:03,741 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: leolani2_know_fred [person_->_])
2021-12-09 10:33:03,835 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-12-09 10:33:03,887 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Overlaps: 1 subject overlaps: e.g. piek on December,2021 about piek - 0 object overlaps: e.g. ''
2021-12-09 10:33:06,687 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred on December,2021 about UNDERSPECIFIED
2021-12-09 10:33:06,791 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 ga

## Continue the conversation with another speaker

In [19]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, initial_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
    scenario_ctrl.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario_ctrl,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain,
                           print_details)

    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, reply, AGENT)
    scenario_ctrl.append_signal(textSignal)

Leolani2: Do you have any gossip?



 I am from Rotterdam


Fred: I am from Rotterdam
2021-12-09 10:35:12,829 -     INFO - cltl.triple_extraction.api.Chat (Fred)              001 -       Fred: "I am from Rotterdam"
initial triple: {'subject': 'I', 'predicate': 'am', 'object': 'from-Rotterdam'}
2021-12-09 10:35:12,898 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_be-from_rotterdam [person_->_location])
2021-12-09 10:35:12,944 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-12-09 10:35:15,395 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred on December,2021 about POSITIVE
2021-12-09 10:35:15,489 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. live-in location - 15 gaps as object: e.g. be-friends-with person
2021-12-09 10:35:15,540 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 0 gaps as subject: e.g. '' - 5 gaps as object: e.g. manufacture-in robot
Leolani2: I would l


 I like cats


Fred: I like cats
2021-12-09 10:35:19,715 -     INFO - cltl.triple_extraction.api.Chat (Fred)              002 -       Fred: "I like cats"
initial triple: {'subject': 'I', 'predicate': 'like', 'object': 'cats'}
2021-12-09 10:35:19,748 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_like_cats [person_->_animal])
2021-12-09 10:35:19,885 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Overlaps: 0 subject overlaps: e.g. '' - 1 object overlaps: e.g. piek on December,2021 about piek
2021-12-09 10:35:22,486 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: fred on December,2021 about POSITIVE
2021-12-09 10:35:22,542 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-member-of institution - 15 gaps as object: e.g. know agent
2021-12-09 10:35:22,596 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 0 gaps as subject: e.g. '' - 2 gaps as object: e.g. own agent
Leolani2: I am cur

...

Fred: Who does have cats
2021-12-09 10:37:04,155 -     INFO - cltl.triple_extraction.api.Chat (Fred)              011 -       Fred: "Who does have cats"
2021-12-09 10:37:04,158 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: cats_be_? [animal_->_])
Leolani2: I may be ignorant, bu tI don't know.I am sorry, I have no idea



 stop


Fred: stop
2021-12-09 10:37:08,202 -     INFO - cltl.triple_extraction.api.Chat (Fred)              012 -       Fred: "stop"


Couldn't parse input


Leolani2: Be that as it may, still may it be as it may be


## Save the scenario data

In [20]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook